In [1]:
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/bjn_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/eng_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/ind_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/jav_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zsm_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zho_Hans.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/tam_Taml.dev

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
with open('tam_Taml.dev') as fopen:
    en = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
en_, ms_ = [], []
for i in range(len(en)):
    if len(en[i]) and len(ms[i]):
        en_.append(en[i])
        ms_.append(ms[i])

In [4]:
len(en_), len(ms_)

(997, 997)

In [5]:
from tqdm import tqdm
import requests
import os
import json

In [8]:
# !rm -rf base-ta-ms
!mkdir base-ta-ms

mkdir: cannot create directory ‘base-ta-ms’: File exists


In [9]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-base-malaysian-translation-v2')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/nanot5-base-malaysian-translation-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [10]:
_ = model.cuda()

In [11]:
all_special_ids = [0, 1, 2]

In [14]:
input_ids = tokenizer.encode(f'terjemah ke Melayu: {en_[0]}{tokenizer.eos_token}', return_tensors = 'pt')
outputs = model.generate(input_ids.cuda(), max_length = 1024, num_beams=5, early_stopping=True)
outputs = [i for i in outputs[0] if i not in all_special_ids]
print(tokenizer.decode(outputs, spaces_between_special_tokens = False).strip())

Pada bulan Ogos, saintis Stanford University of Massachusetts mengumumkan penemuan alat pengaturcaraan baharu yang boleh diurutkan mengikut jenis sel: 1 SG digunakan untuk setiap 1 unit.


In [15]:
for i in tqdm(range(len(ms_))):
    filename = os.path.join('base-ta-ms', f'{i}.json')
    
    if os.path.exists(filename):
        continue
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    input_ids = tokenizer.encode(f'terjemah ke Melayu: {en_[i]}{tokenizer.eos_token}', return_tensors = 'pt')
    outputs = model.generate(input_ids.cuda(), max_length = 1024, num_beams=5, early_stopping=True)
    outputs = [i for i in outputs[0] if i not in all_special_ids]
    r = tokenizer.decode(outputs, spaces_between_special_tokens = False).strip()

    with open(filename, 'w') as fopen:
        json.dump({'text': ms_[i], 'r': r}, fopen)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 997/997 [02:58<00:00,  5.58it/s]


In [16]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [17]:
predicted = []
for i in range(len(ms_)):
    filename = os.path.join('base-ta-ms', f'{i}.json')
    with open(filename) as fopen:
        d = json.load(fopen)
    predicted.append(d['r'])

In [18]:
score = chrf.corpus_score(predicted, [ms_])
score

chrF2++ = 40.47